# Traffic Tune - Optimizing Traffic Signals with Reinforcement Learning

## Introduction

Welcome to the Traffic Tune POC notebook. Our project focused on optimizing traffic signal control using reinforcement learning. Traffic congestion is a major problem in urban areas, leading to increased travel times, fuel consumption, and pollution. Traditional traffic signal control systems often struggle to adapt to dynamic traffic conditions, resulting in suboptimal traffic flow.

Traffic Tune is a recommendation system that leverages reinforcement learning to dynamically adjust traffic signals at intersections. By learning from traffic patterns in real-time, Traffic Tune aims to improve traffic flow, reduce congestion, and enhance overall transportation efficiency.

In this POC, we will demonstrate how to train a reinforcement learning agent to optimize traffic signal control in a simulated environment. We will use the SUMO (Simulation of Urban MObility) traffic simulation tool and the Stable Baselines3 library to train a Deep Q-Network (DQN) agent to learn an optimal traffic signal control policy.


# Setup and Installations

In [1]:
import env_manager as env_manager
import ppo_trainer as ppo_trainer
import dqn_trainer as dqn_trainer

In [2]:
def chain_training(manager: env_manager, generator: env_manager.EnvManager.env_generator, algo_agent, running_result: list):
    if len(running_result) != 0: 
        # take the best config from the previous training 
        best = running_result[-1].get_best_result("env_runners/episode_reward_max", "max")
        
        # Initialize the environment manager with new route file
        rou, csv = next(generator)
        manager.initialize_env(rou, csv)
        
        # continue the training with the best config
        algo_agent.config = best.config
        algo_agent.build_config()
    
    result = algo_agent.train()
    
    return result

def training(num_intersection: int, experiment_type: str, algo_config: str, env_config: str, num_training_cycles: int):
    running_result = []
    algo_agent = None
    sumo_type = "SingleAgent"
    algo_type = experiment_type.split("_")
    
    if experiment_type.__contains__("Multi"):
        sumo_type = "MultiAgent"
    
    # Initialize the environment manager
    manager = env_manager.EnvManager(f"{sumo_type}Environment", env_config, json_id=f"intersection_{num_intersection}")
    generator = manager.env_generator(f"Nets/intersection_{num_intersection}/route_xml_path_intersection_{num_intersection}.txt", algo_name=algo_type[0])
    
    # Initialize the environment manager with new route file
    rou, csv = next(generator)
    manager.initialize_env(rou, csv)
    
    # Initialize the Algo agent
    if algo_config.startswith("ppo"):
        ppo_agent = ppo_trainer.PPOTrainer(config_path=algo_config, env_manager=manager, experiment_type=experiment_type)
        ppo_agent.build_config()
        algo_agent = ppo_agent
        
    elif algo_config.startswith("dqn"):
        dqn_agent = dqn_trainer.DQNTrainer(config_path=algo_config, env_manager=manager, experiment_type=experiment_type)
        dqn_agent.build_config()
        algo_agent = dqn_agent

    for i in range(num_training_cycles):
        chain_result = chain_training(manager=manager, generator=generator, algo_agent=algo_agent, running_result=running_result)
        if chain_result is not None:
            running_result.append(chain_result)
    
    return running_result

In [3]:
num_intersection_to_train = 3  # Choose which intersection you want to train

# Choose the experiment_type:
# PPO_SingleAgent | PPO_MultiAgent | DQN_SingleAgent | DDQN_SingleAgent | DQN_MultiAgent | DDQN_MultiAgent
experiment_type = "PPO_SingleAgent"  

num_training_cycles = 1

env_config_file_path = "env_config.json"

ppo_config_file_path = "ppo_config.json"

dqn_config_file_path = "dqn_config.json"

In [4]:
results = training(num_intersection=num_intersection_to_train, experiment_type=experiment_type, algo_config=ppo_config_file_path, env_config=env_config_file_path, num_training_cycles=num_training_cycles)

2024-07-25 00:23:11,833	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!
2024-07-25 00:23:11,834	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!
2024-07-25 00:23:11,834	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_learner_workers` has been deprecated. Use `AlgorithmConfig.num_learners` instead. This will raise an error in the future!
2024-07-25 00:23:11,834	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_learner_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_learner` instead. This will raise an error in the future!
2024-07-25 00:23:11,834	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus

(PPO pid=4653) 2024-07-25 00:23:13,416	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_env_runner` instead. This will raise an error in the future!
(PPO pid=4653) 2024-07-25 00:23:13,416	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_gpus_per_worker` has been deprecated. Use `AlgorithmConfig.num_gpus_per_env_runner` instead. This will raise an error in the future!
(PPO pid=4653) 2024-07-25 00:23:13,416	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_learner_workers` has been deprecated. Use `AlgorithmConfig.num_learners` instead. This will raise an error in the future!
(PPO pid=4653) 2024-07-25 00:23:13,416	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.num_cpus_per_learner_worker` has been deprecated. Use `AlgorithmConfig.num_cpus_per_learner` instead. This will raise an error in the future!
(PPO pid=4653) 2024-07-25 00:23:13,416

(RolloutWorker pid=4654)  Retrying in 1 seconds


(RolloutWorker pid=4654) 2024-07-25 00:23:16,101	INFO policy.py:1272 -- Policy (worker=1) running on CPU.
(RolloutWorker pid=4654) 2024-07-25 00:23:16,101	INFO torch_policy_v2.py:111 -- Found 0 visible cuda devices.


(RolloutWorker pid=4654) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 0 ACT 0 BUF 0)                      


...

Step #3100.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 3755 ACT 51 BUF 1)                0ms, vehicles TOT 3520 ACT 61 BUF 


(RolloutWorker pid=4654) Warning: Vehicle 'f_5.1050' performs emergency braking on lane '32299300_2' with decel=9.00, wished=4.50, severity=1.00, time=3562.00.
(RolloutWorker pid=4654) Warning: Vehicle 'f_5.1050' performs emergency stop at the end of lane '32299300_2' because of a red traffic light (decel=-21.11, offset=12.11), time=3562.00.


Step #3500.00 (1ms ~= 1000.00*RT, ~48000.00UPS, TraCI: 7ms, vehicles TOT 4235 ACT 48 BUF 03867 ACT 41 BUF 1)                
(RolloutWorker pid=4654) Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 28ms, vehicles TOT 4353 ACT 44 BUF 1)               
(RolloutWorker pid=4654)  Retrying in 1 seconds


2024-07-25 00:23:41,214	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/eviat/Desktop/Final_Project/Traffic_Tune_Project/Outputs/Training/intersection_3/saved_agent/PPO_2024-07-25_00-23-11' in 0.0058s.
(PPO pid=4653) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/eviat/Desktop/Final_Project/Traffic_Tune_Project/Outputs/Training/intersection_3/saved_agent/PPO_2024-07-25_00-23-11/PPO_PPO_ee2ea_00000_0_2024-07-25_00-23-11/checkpoint_000002)
2024-07-25 00:23:41,650	INFO tune.py:1041 -- Total run time: 29.85 seconds (29.38 seconds for the tuning loop).


Step #5.00 (0ms ?*RT. ?UPS, TraCI: 664ms, vehicles TOT 2 ACT 2 BUF 0)                     ACT 2 BUF 0)                      


In [ ]:
print(results)